In [2]:
!pip install yfinance


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
      --------------------------------------- 0.0/3.0 MB 991.0 kB/s eta 0:00:04
     ---- ----------------------------------- 0.3/3.0 MB 5.3 MB/s eta 0:00:01
     --------------------- ------------------ 1.6/3.0 MB 12.9 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 19.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/123.4 kB ? eta -:--:--
   ---------------------------------------- 123.4/123.4 kB 7.1 MB/s eta 0:00:00
   ---------------------------

In [7]:
# ============================================================
# ÉTAPE 1 — CHARGEMENT DES DONNÉES SPY + EXPIRATIONS DISPONIBLES
# ============================================================

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import norm

# Charger les données SPY
ticker = yf.Ticker("SPY")

# Voir les dates d'expiration disponibles
expirations = ticker.options
print("Dates disponibles :", expirations)


Dates disponibles : ('2025-12-09', '2025-12-10', '2025-12-11', '2025-12-12', '2025-12-15', '2025-12-16', '2025-12-17', '2025-12-18', '2025-12-19', '2025-12-26', '2025-12-31', '2026-01-02', '2026-01-09', '2026-01-16', '2026-01-23', '2026-01-30', '2026-02-20', '2026-02-27', '2026-03-20', '2026-03-31', '2026-04-30', '2026-05-29', '2026-06-18', '2026-06-30', '2026-09-18', '2026-09-30', '2026-12-18', '2027-01-15', '2027-03-19', '2027-06-17', '2027-12-17', '2028-01-21')


In [8]:
# ============================================================
# ÉTAPE 2 — CHOISIR UNE DATE D'EXPIRATION AVEC T > 0.01
# ============================================================

today = datetime.now()

valid_exp = None
for exp in expirations:
    exp_date = datetime.strptime(exp, "%Y-%m-%d")
    T = (exp_date - today).days / 365
    if T > 0.01:
        valid_exp = exp
        break

print("Expiration utilisée :", valid_exp)

# Charger la chaîne d’options SPY (calls + puts)
opt_chain = ticker.option_chain(valid_exp)
calls = opt_chain.calls.copy()
puts = opt_chain.puts.copy()

calls["type"] = "call"
puts["type"] = "put"

df = pd.concat([calls, puts], ignore_index=True)
df.head()


Expiration utilisée : 2025-12-15


,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,type
0,SPY251215C00500000,2025-12-08 19:06:16+00:00,500.0,182.49,182.50,185.18,-2.419998,-1.308744,12.0,1,0.942383,True,REGULAR,USD,call
1,SPY251215C00585000,2025-12-03 20:01:28+00:00,585.0,100.64,97.59,100.27,0.000000,0.000000,NaN,4,0.534673,True,REGULAR,USD,call
2,SPY251215C00590000,2025-12-04 20:36:52+00:00,590.0,93.83,92.60,95.28,0.000000,0.000000,1.0,1,0.511724,True,REGULAR,USD,call
3,SPY251215C00610000,2025-12-01 20:59:44+00:00,610.0,70.89,72.64,75.31,0.000000,0.000000,NaN,3,0.568608,True,REGULAR,USD,call
4,SPY251215C00620000,2025-12-04 15:44:07+00:00,620.0,65.30,62.66,65.34,0.000000,0.000000,NaN,3,0.507085,True,REGULAR,USD,call


In [9]:
# ============================================================
# ÉTAPE 3 — CLEANING DES DONNÉES + CALCUL DES VARIABLES
# ============================================================

# Prix mid
df["mid"] = (df["bid"] + df["ask"]) / 2

# Enlever les options inutilisables
df = df[df["mid"] > 0]

# Spot price actuel
spot = ticker.history(period="1d")["Close"].iloc[-1]
df["spot"] = spot

# Temps à maturité T en années
exp_date = datetime.strptime(valid_exp, "%Y-%m-%d")
df["T"] = (exp_date - today).days / 365

# Sélection colonnes utiles
df = df[["strike", "type", "mid", "spot", "T"]]

df.head()


,strike,type,mid,spot,T
0,500.0,call,183.840,683.630005,0.016438
1,585.0,call,98.930,683.630005,0.016438
2,590.0,call,93.940,683.630005,0.016438
3,610.0,call,73.975,683.630005,0.016438
4,620.0,call,64.000,683.630005,0.016438


In [10]:
# ============================================================
# ÉTAPE 4 — FONCTION BLACK-SCHOLES
# ============================================================

def black_scholes(option_type, S, K, r, sigma, T):
    if T <= 0:
        return np.nan
    
    d1 = (np.log(S/K) + (r + 0.5 * sigma**2) * T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)


In [11]:
# ============================================================
# ÉTAPE 5 — CALCUL DE LA VOLATILITÉ IMPLICITE (IV)
# ============================================================

def implied_vol(option_type, price, S, K, r, T, tol=1e-6, max_iter=100):
    if T <= 0 or price <= 0:
        return np.nan

    sigma = 0.3  # Valeur initiale

    for i in range(max_iter):
        model_price = black_scholes(option_type, S, K, r, sigma, T)

        # Vega
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        vega = S * norm.pdf(d1) * np.sqrt(T)

        if vega == 0:
            return np.nan

        sigma_new = sigma - (model_price - price) / vega

        if abs(sigma_new - sigma) < tol:
            return sigma_new

        sigma = sigma_new

    return np.nan


In [12]:
# ============================================================
# ÉTAPE 6 — BOUCLE DE CALCUL DE IV POUR CHAQUE OPTION
# ============================================================

r = 0.02  # Taux sans risque
ivs = []

for idx, row in df.iterrows():
    iv = implied_vol(
        row["type"],
        row["mid"],
        row["spot"],
        row["strike"],
        r,
        row["T"]
    )
    ivs.append(iv)

df["IV"] = ivs

df.head()


,strike,type,mid,spot,T,IV
0,500.0,call,183.840,683.630005,0.016438,NaN
1,585.0,call,98.930,683.630005,0.016438,0.505086
2,590.0,call,93.940,683.630005,0.016438,0.484983
3,610.0,call,73.975,683.630005,0.016438,0.398291
4,620.0,call,64.000,683.630005,0.016438,0.355365


In [ ]:
# ============================================================
# ÉTAPE 7 — VISUALISATION : VOLATILITY SMILE
# ============================================================

import matplotlib.pyplot as plt

# Garder uniquement les options avec IV valide
df_valid = df.dropna(subset=["IV"])

plt.figure(figsize=(10,6))
plt.scatter(df_valid["strike"], df_valid["IV"], color="blue", alpha=0.6)
plt.plot(df_valid["strike"], df_valid["IV"], color="black", linewidth=1)

plt.title("Volatility Smile - SPY Options")
plt.xlabel("Strike Price")
plt.ylabel("Implied Volatility (IV)")
plt.grid(True)
plt.show()
